In [32]:
import sys
import os
import nltk

In [33]:
class classproperty(object):
    '''
    This is a helper that will be used as a decorator.
    Class property handling doesn't natively exist in Python.
    '''
    def __init__(self, getter):
        self.getter= getter
    def __get__(self, instance, owner):
        return self.getter(owner)

In [34]:
class Book():
    _all = []
    
    @classmethod
    def add_to_all(cls,obj):
        if obj._str == None: #it's a book
            if obj.title not in [i.title for i in cls._all if i.author == obj.author]:
                cls._all.append(obj)
            else:
                print('Entry already exists.')
        elif obj._str not in [i._str for i in cls._all]: #it's a sample and it's not read in yet
            cls._all.append(obj)
        else:
            print('Entry already exists.')
        
    def __init__(self,author,title,_str=None):
        self._str = _str
        self.fpath = author + '/' + title
        self.title = title
        self.author = author
        self.__class__.add_to_all(self)
    
    @classproperty
    def all(cls):
        return cls._all
    
    def read(self):
        with open('./Authors/' + self.fpath + '.txt',encoding='iso-8859-1') as f:
            self.text = f.read()
        #need to validate this
        self.text = self.text.split('Falls Sie Fehler finden bitte bei DigBib.Org melden.')[1]
        self.text = self.text.replace('\n',' ')
    def preprocess(self):
        #remove stopwords and certain characters
        pass

In [35]:
#inherit preprocessing
class Sample():
    def __init__(self,_str):
        return Book(self,None,None,_str)
    def read(self):
        self.text = self._str

In [36]:
def get_books():
    for author in os.listdir('Authors'):
        if author != '.DS_Store': #not an author i'm familiar with
            for book in os.listdir('Authors/' + author):
                Book(author,book.split('.')[0])

In [37]:
def read_books():
    for book in Book.all:
        book.read()

In [38]:
get_books()
read_books()

In [40]:
Book.all[20].text[0:300]

'  Knabenmärchen  aus Dichtung und Wahrheit (1811)  Mir träumte neulich in der Nacht vor Pfingstsonntag, als stünde ich vor einem Spiegel und beschäftigte mich mit den neuen Sommerkleidern, welche mir die lieben Eltern auf das Fest hatten machen lassen. Der Anzug bestand, wie ihr wißt, in Schuhen von'